In [136]:
import requests
import praw
import pandas as pd
import json
import time
import re
from typing import List, Dict, Optional
from html import unescape

# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

In [137]:
# from praw.models import MoreComments #This import is needed is used only if we want to limit the comments we scrape

# #URL to access the app needed to scrape the data off the Berkeley subreddit
# # https://www.reddit.com/prefs/apps

#This is the tutorial I used to set up the web scraping using PRAW
#https://www.geeksforgeeks.org/python/scraping-reddit-using-python/

# #PLAN
# #Scrape the data off the Berkeley subreddit about consulting clubs
# #Organize that data using the pandas library
# #Create chatbot that utilizes natural language processing that will give users feedback
# #about each consulting club here at Berkeley. 

reddit_read_only = praw.Reddit(client_id = "QlBfNfxQ3e_MGP9RkaOQig",
                               client_secret = "SpLjOwYdQPU4z1wqcXBjVl_7DnUIZg",
                               user_agent = "Berkeley_Consulting")

subreddit = reddit_read_only.subreddit("berkeley")

In [138]:
consulting_posts = subreddit.search('clubs', limit=None)

posts_dict = {'Title': [], 'Post Text': [], 'ID': [], 'Score': [], 'Total Comments': [], 'Post URL': []}
comments_dict = {'Comment': [], 'Score': []}

In [139]:
for post in consulting_posts:
    if len(post.selftext) > 20:
        #The title of the post
        posts_dict['Title'].append(post.title)
        #The text inside of the post
        posts_dict['Post Text'].append(post.selftext)
        #Unique ID of each post
        posts_dict['ID'].append(post.id)
        #The scoure of a post
        posts_dict['Score'].append(post.score)
        #Total number of comments inside the post
        posts_dict['Total Comments'].append(post.num_comments)
        #URL of each post
        posts_dict['Post URL'].append(post.url)
    else:
        continue

consulting_club_posts = pd.DataFrame(posts_dict)
# print(consulting_club_posts)
consulting_club_posts

,Title,Post Text,ID,Score,Total Comments,Post URL
0,Looking for people to start a bridge troll clu...,"Hello, \n\nI was wondering if anyone else woul...",1nfcxn2,261,36,https://www.reddit.com/r/berkeley/comments/1nf...
1,Heartbreaking club experience as a freshman,I applied to 9 tech clubs (i’m an eecs major) ...,1nh1egm,130,52,https://www.reddit.com/r/berkeley/comments/1nh...
2,Berkeley club decisions today got me like:,[Berkeley club decisions today got me like:](h...,1na5tea,72,50,https://www.reddit.com/r/berkeley/comments/1na...
3,Potential nepotism in prestigious clubs is sic...,Title. So exhausted of this kinda stuff. Heard...,1nbpwed,106,36,https://www.reddit.com/r/berkeley/comments/1nb...
4,Asian monoculture in consulting clubs,Over the past month on this subreddit there ha...,16o5u8z,268,112,https://www.reddit.com/r/berkeley/comments/16o...
...,...,...,...,...,...,...
204,Incoming freshman curious about consulting/bus...,Hey guys! I am super excited to come to Berkel...,1ebn5q3,0,7,https://www.reddit.com/r/berkeley/comments/1eb...
205,Are there any clubs on campus for students who...,Basically what the title says! Or if there are...,1h0w654,3,0,https://www.reddit.com/r/berkeley/comments/1h0...
206,What are some fun clubs on campus?,Anyone have any recommendations?,198ng9e,15,16,https://www.reddit.com/r/berkeley/comments/198...
207,[Club Recruitment Megathread] Have a club that...,Here is the format you should follow to ensure...,i9yljd,80,46,https://www.reddit.com/r/berkeley/comments/i9y...


In [140]:
#Need to figure out a way to get the URL of each and every post
for i in posts_dict['Post URL']:
    if 'comments' in i:
        submission = reddit_read_only.submission(url = i)
    else:
        continue
    for comment in submission.comments:
        if len(comment.body) < 20:
            continue
        comments_dict['Comment'].append(comment.body)
        comments_dict['Score'].append(comment.score)

# print(post_comments[0]) #This only prints one comment right now for one URL. Need to get as many comments as possible for one given URL.

comments_df = pd.DataFrame(comments_dict)
comments_df

#Generate the CSV file that contains all the posts about Berkeley clubs
# comments_df.to_csv('my_dataframe.csv', index=False) # index=False prevents writing the DataFrame index as a column

#They are not marked in any way so they are just random pieces of information
#Need to find a way to associate either the post title or the post URL with the post comments

,Comment,Score
0,Don’t forget riddles,63
1,"if you’re deadass, i will absolutely join",49
2,"I'm actually serious about starting it, if som...",50
3,"definitely don’t go barefoot under a bridge, t...",15
4,Do we get to eat billy goats?,13
...,...,...
1043,I’d say actually practice case interviews. A l...,9
1044,"Lets see, how do you look? Physiognomy will de...",2
1045,"idk if this applies to all consulting clubs, b...",2
1046,Follow the 2 rules: \n\n1. Be attractive \n2. ...,3


In [141]:
new_df = pd.concat([consulting_club_posts['Post Text'], comments_df['Comment']], axis=1)
new_df

,Post Text,Comment
0,"Hello, \n\nI was wondering if anyone else woul...",Don’t forget riddles
1,I applied to 9 tech clubs (i’m an eecs major) ...,"if you’re deadass, i will absolutely join"
2,[Berkeley club decisions today got me like:](h...,"I'm actually serious about starting it, if som..."
3,Title. So exhausted of this kinda stuff. Heard...,"definitely don’t go barefoot under a bridge, t..."
4,Over the past month on this subreddit there ha...,Do we get to eat billy goats?
...,...,...
1043,NaN,I’d say actually practice case interviews. A l...
1044,NaN,"Lets see, how do you look? Physiognomy will de..."
1045,NaN,"idk if this applies to all consulting clubs, b..."
1046,NaN,Follow the 2 rules: \n\n1. Be attractive \n2. ...


In [142]:
# #Loading the stored dataframes from the previous notebook using the pickle module
# consulting_club_posts.to_pickle('consulting_club_posts.pkl')
# comments_df.to_pickle('comments_df.pkl')

In [143]:
#!/usr/bin/env python3
"""
CalLink Hybrid Scraper
Uses API for organization list + Selenium for detailed page scraping
"""
def strip_html(text: str) -> str:
    """Remove HTML tags and clean up text"""
    if not text or not isinstance(text, str):
        return ''
    
    text = re.sub(r'<[^>]+>', '', text)
    text = unescape(text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text

In [144]:
def setup_driver():
    """Setup Chrome driver with appropriate options"""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in background
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36')
    
    driver = webdriver.Chrome(options=options)
    return driver

In [145]:
def fetch_organization_list() -> List[Dict]:
    """Fetch the list of all organizations from the API"""
    base_url = "https://callink.berkeley.edu/api/discovery/search/organizations"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36',
        'Accept': 'application/json',
        'Referer': 'https://callink.berkeley.edu/Organizations',
    }
    
    all_organizations = []
    skip = 0
    top = 100
    
    print("📋 Fetching organization list from CalLink API...")
    print("=" * 70)
    
    while True:
        params = {
            'top': top,
            'skip': skip,
            'orderBy[0]': 'UpperName asc',
            'query': '',
        }
        
        try:
            print(f"  Fetching organizations {skip + 1} to {skip + top}...", end=" ")
            response = requests.get(base_url, params=params, headers=headers, timeout=15)
            
            if response.status_code != 200:
                print(f"❌ Error: Status code {response.status_code}")
                break
            
            data = response.json()
            
            if isinstance(data, dict):
                orgs = data.get('value', data.get('items', data.get('results', [])))
            elif isinstance(data, list):
                orgs = data
            else:
                print("❌ Unexpected response format")
                break
            
            if not orgs:
                print("✓ Done")
                break
            
            all_organizations.extend(orgs)
            print(f"✓ Got {len(orgs)} (Total: {len(all_organizations)})")
            
            if len(orgs) < top:
                break
            
            skip += top
            time.sleep(0.3)
            
        except Exception as e:
            print(f"❌ Error: {e}")
            break
    
    print(f"\n✓ Found {len(all_organizations)} organizations total\n")
    return all_organizations

In [146]:
def scrape_organization_page_selenium(driver, org_url: str, org_name: str) -> Optional[Dict]:
    """
    Use Selenium to scrape detailed information from an organization page
    """
    try:
        driver.get(org_url)
        
        # Wait for React app to load and render content
        try:
            # First wait for React app container
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.ID, "react-app"))
            )
            # Then wait for MuiGrid content (main content container)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "MuiGrid-root"))
            )
            # Give React extra time to populate all content
            time.sleep(7)
        except TimeoutException:
            return None
        
        # Parse with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        details = {}
        
        # Get ALL text from the main React app
        react_app = soup.find('div', id='react-app')
        if react_app:
            # Get all text content, which includes the full description
            all_text = strip_html(react_app.get_text(separator=' '))
            
            # Store full text (limit to reasonable length)
            if len(all_text) > 100:
                details['full_page_content'] = all_text[:10000]
            
            # Try to extract structured sections if possible
            # Look for any div with substantial text content
            text_divs = react_app.find_all('div')
            long_texts = []
            for div in text_divs:
                text = strip_html(div.get_text())
                # If this div has substantial unique text, it might be the description
                if len(text) > 200 and text not in long_texts:
                    long_texts.append(text)
            
            # The longest text is likely the full description
            if long_texts:
                details['full_description'] = max(long_texts, key=len)[:5000]
        
        # Extract emails
        emails = soup.find_all('a', href=re.compile(r'mailto:'))
        if emails:
            details['contact_emails'] = ', '.join(set([e.get('href', '').replace('mailto:', '') for e in emails]))
        
        # Extract social media links
        social_platforms = {
            'facebook_url': r'facebook\.com',
            'instagram_url': r'instagram\.com',
            'twitter_url': r'twitter\.com|x\.com',
            'linkedin_url': r'linkedin\.com',
            'discord_url': r'discord\.(gg|com)',
            'tiktok_url': r'tiktok\.com',
        }
        
        for platform, pattern in social_platforms.items():
            link = soup.find('a', href=re.compile(pattern))
            if link:
                details[platform] = link.get('href', '')
        
        # Extract external website
        website = soup.find('a', href=re.compile(r'^https?://(?!callink\.berkeley)'))
        if website:
            url = website.get('href', '')
            # Make sure it's not a social media link we already captured
            if not any(re.search(pattern, url) for pattern in social_platforms.values()):
                details['external_website'] = url
        
        # Try to find meeting information
        meeting_keywords = ['meet', 'when', 'where', 'time', 'location']
        for keyword in meeting_keywords:
            elem = soup.find(string=re.compile(keyword, re.I))
            if elem:
                # Get parent and surrounding text
                parent = elem.find_parent()
                if parent:
                    meeting_text = strip_html(parent.get_text())
                    if len(meeting_text) > 10:
                        details['meeting_info'] = meeting_text[:500]
                        break
        
        return details if details else None
        
    except Exception as e:
        return None

In [147]:
def scrape_all_organizations_detailed(driver, org_list: List[Dict], max_orgs: Optional[int] = None) -> List[Dict]:
    """
    Visit each organization's page and collect detailed information using Selenium
    """
    print("🔍 Scraping detailed information from each organization page...")
    print("=" * 70)
    
    if max_orgs:
        print(f"⚠️  TEST MODE: Only scraping first {max_orgs} organizations\n")
        org_list = org_list[:max_orgs]
    
    enriched_orgs = []
    failed_count = 0
    
    for idx, org in enumerate(org_list, 1):
        # Get basic info from API response
        org_data = {
            'name': strip_html(org.get('Name', org.get('name', ''))),
            'short_description': strip_html(org.get('Description', org.get('description', ''))),
            'status': strip_html(org.get('Status', '')),
            'api_email': org.get('Email', org.get('email', '')),
        }
        
        # Get categories
        if 'CategoryNames' in org:
            cats = org['CategoryNames']
            org_data['categories'] = ', '.join([strip_html(str(c)) for c in cats]) if isinstance(cats, list) else strip_html(str(cats))
        
        # Build profile URL
        website_key = org.get('WebsiteKey', '')
        if website_key:
            org_data['profile_url'] = f"https://callink.berkeley.edu/organization/{website_key}"
        else:
            org_data['profile_url'] = ''
        
        # Scrape detailed page with Selenium
        print(f"[{idx}/{len(org_list)}] {org_data['name'][:50]}...", end=" ")
        
        if org_data['profile_url']:
            details = scrape_organization_page_selenium(driver, org_data['profile_url'], org_data['name'])
            
            if details:
                org_data.update(details)
                print("✓")
            else:
                print("⚠️  (basic info only)")
                failed_count += 1
        else:
            print("❌ (no URL)")
            failed_count += 1
        
        enriched_orgs.append(org_data)
        
        # Small delay between requests
        time.sleep(0.5)
        
        # Progress update
        if idx % 25 == 0:
            print(f"\n  Progress: {idx}/{len(org_list)} completed ({failed_count} with limited info)\n")
    
    print(f"\n✓ Completed! Successfully scraped {len(enriched_orgs) - failed_count}/{len(enriched_orgs)} organizations")
    print(f"  ({failed_count} organizations had limited/no detailed info)\n")
    
    return enriched_orgs

In [148]:
def save_data(orgs: List[Dict], prefix: str = 'callink_complete'):
    """Save scraped data to CSV and JSON"""
    if not orgs:
        print("❌ No data to save")
        return
    
    df = pd.DataFrame(orgs)
    
    # Save CSV
    csv_file = f'{prefix}.csv'
    df.to_csv(csv_file, index=False, encoding='utf-8')
    print(f"✓ Saved CSV: {csv_file}")
    
    # Save JSON
    json_file = f'{prefix}.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(orgs, f, indent=2, ensure_ascii=False)
    print(f"✓ Saved JSON: {json_file}")
    
    # Print summary
    print("\n" + "=" * 70)
    print("📊 DATA SUMMARY")
    print("=" * 70)
    print(f"Total organizations: {len(df)}")
    print(f"\nColumns ({len(df.columns)}):")
    for col in df.columns:
        non_null = df[col].notna().sum()
        pct = (non_null / len(df)) * 100
        print(f"  • {col}: {non_null}/{len(df)} ({pct:.1f}%)")
    
    print(f"\nSample organizations:")
    print(df[['name', 'categories']].head(5).to_string(index=False))

In [149]:
def main():
    print("\n" + "=" * 70)
    print("🐻 BERKELEY CALLINK SELENIUM SCRAPER")
    print("=" * 70)
    print("Uses API for list + Selenium for detailed page scraping")
    print("=" * 70 + "\n")
    
    # Step 1: Get organization list from API
    org_list = fetch_organization_list()
    
    if not org_list:
        print("❌ Failed to fetch organization list")
        print("\nMake sure you have internet access and CalLink is available.")
        return
    
    print(f"Found {len(org_list)} organizations.")
    print("\n⚙️  TEST MODE: Will scrape first 10 organizations")
    print("    (To scrape all, change max_orgs=10 to max_orgs=None in the code)\n")
    
    max_orgs = None  # Change to None for full scrape
    
    if max_orgs:
        print(f"🧪 Scraping first {max_orgs} organizations\n")
    else:
        print(f"🚀 Scraping all {len(org_list)} organizations")
        print(f"⏱️  Estimated time: {len(org_list) * 1.5 / 60:.0f}-{len(org_list) * 2 / 60:.0f} minutes\n")
    
    # Step 2: Setup Selenium
    print("🌐 Starting Chrome browser...")
    driver = setup_driver()
    
    try:
        # Step 3: Scrape detailed info
        enriched_orgs = scrape_all_organizations_detailed(driver, org_list, max_orgs=max_orgs)
        
        # Step 4: Save data
        print("\n💾 Saving data...")
        prefix = 'callink_test' if max_orgs else 'callink_complete'
        save_data(enriched_orgs, prefix=prefix)
        
        print("\n" + "=" * 70)
        print("✅ SCRAPING COMPLETE!")
        print("=" * 70)
        
        if max_orgs:
            print("\n💡 TIP: To scrape all organizations, change:")
            print("   max_orgs = 10")
            print("   to")
            print("   max_orgs = None")
        
    finally:
        print("\n🔒 Closing browser...")
        driver.quit()


if __name__ == "__main__":
    main()


🐻 BERKELEY CALLINK SELENIUM SCRAPER
Uses API for list + Selenium for detailed page scraping

📋 Fetching organization list from CalLink API...
  Fetching organizations 1 to 100... ✓ Got 100 (Total: 100)
  Fetching organizations 101 to 200... ✓ Got 100 (Total: 200)
  Fetching organizations 201 to 300... ✓ Got 100 (Total: 300)
  Fetching organizations 301 to 400... ✓ Got 100 (Total: 400)
  Fetching organizations 401 to 500... ✓ Got 100 (Total: 500)
  Fetching organizations 501 to 600... ✓ Got 100 (Total: 600)
  Fetching organizations 601 to 700... ✓ Got 100 (Total: 700)
  Fetching organizations 701 to 800... ✓ Got 100 (Total: 800)
  Fetching organizations 801 to 900... ✓ Got 100 (Total: 900)
  Fetching organizations 901 to 1000... ✓ Got 100 (Total: 1000)
  Fetching organizations 1001 to 1100... ✓ Got 100 (Total: 1100)
  Fetching organizations 1101 to 1200... ✓ Got 100 (Total: 1200)
  Fetching organizations 1201 to 1300... ✓ Got 100 (Total: 1300)
  Fetching organizations 1301 to 1400... ✓

In [150]:
# new_df.to_csv('posts_and_comments.csv', index=False)